In [2]:
!pip install transformers[torch] accelerate -U


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizer,AutoTokenizer

# Load your data
df = pd.read_csv('/kaggle/input/hatedataset/hateDataset.csv')
df.reset_index(drop=True, inplace=True)
# Encode the labels
le = LabelEncoder()
df['label'] = le.fit_transform(df['label'])


# Split the data and reset the indices
train_texts, test_texts, train_labels, test_labels = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)
train_texts.reset_index(drop=True, inplace=True)
test_texts.reset_index(drop=True, inplace=True)
train_labels.reset_index(drop=True, inplace=True)
test_labels.reset_index(drop=True, inplace=True)



In [23]:
df.

534872

In [24]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained('prajjwal1/bert-medium')

# Tokenize the text
train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(test_texts.tolist(), truncation=True, padding=True, max_length=128)

In [25]:
import torch
from torch.utils.data import Dataset

class HateSpeechDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = HateSpeechDataset(train_encodings, train_labels)
test_dataset = HateSpeechDataset(test_encodings, test_labels)


In [26]:
from transformers import BertForSequenceClassification,AutoModelForSequenceClassification, Trainer, TrainingArguments

model = AutoModelForSequenceClassification.from_pretrained('prajjwal1/bert-medium', num_labels=2)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-medium and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [27]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=2,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=200,
    eval_strategy="steps",
    save_steps=200,
    gradient_accumulation_steps=8,
    fp16=True
)


In [28]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

# Train the model
trainer.train()


Step,Training Loss,Validation Loss
200,0.672900,0.612100
400,0.579200,0.533996
600,0.548400,0.494191
800,0.500300,0.489087
1000,0.490700,0.447344
1200,0.402900,0.492378
1400,0.403200,0.443560
1600,0.399500,0.447581
1800,0.388600,0.439837
2000,0.382100,0.435257


TrainOutput(global_step=2056, training_loss=0.47407610592675115, metrics={'train_runtime': 708.5683, 'train_samples_per_second': 92.906, 'train_steps_per_second': 2.902, 'total_flos': 1287631287089664.0, 'train_loss': 0.47407610592675115, 'epoch': 1.9987847855146432})

In [29]:
from sklearn.metrics import classification_report

# Get predictions
preds_output = trainer.predict(test_dataset)
preds = torch.argmax(torch.tensor(preds_output.predictions), axis=1)

# Generate classification report
print(classification_report(test_labels, preds, target_names=le.classes_))


              precision    recall  f1-score   support

        hate       0.78      0.85      0.81      4376
     nothate       0.81      0.72      0.76      3853

    accuracy                           0.79      8229
   macro avg       0.79      0.79      0.79      8229
weighted avg       0.79      0.79      0.79      8229



In [30]:
from sklearn.metrics import classification_report

# Assuming you have already obtained predictions 'preds' and true labels 'test_labels'
# preds = torch.argmax(torch.tensor(preds_output.predictions), axis=1)

# Generate classification report
report = classification_report(test_labels, preds, target_names=le.classes_, output_dict=True)

# Print the classification report
print("Classification Report:")
print(classification_report(test_labels, preds, target_names=le.classes_))

# Extract precision and F1-score for each class and overall
overall_precision = report['accuracy']
overall_f1_score = report['weighted avg']['f1-score']

# Individual precision and F1-score for each class
precision_hate = report['hate']['precision']
precision_nothate = report['nothate']['precision']

f1_score_hate = report['hate']['f1-score']
f1_score_nothate = report['nothate']['f1-score']

# Print overall metrics
print(f"Overall Accuracy: {overall_precision:.2f}")
print(f"Overall F1-Score: {overall_f1_score:.2f}")

# Print individual metrics
print(f"Precision for 'hate': {precision_hate:.2f}")
print(f"Precision for 'nothate': {precision_nothate:.2f}")

print(f"F1-Score for 'hate': {f1_score_hate:.2f}")
print(f"F1-Score for 'nothate': {f1_score_nothate:.2f}")
print("bert-medium")

Classification Report:
              precision    recall  f1-score   support

        hate       0.78      0.85      0.81      4376
     nothate       0.81      0.72      0.76      3853

    accuracy                           0.79      8229
   macro avg       0.79      0.79      0.79      8229
weighted avg       0.79      0.79      0.79      8229

Overall Accuracy: 0.79
Overall F1-Score: 0.79
Precision for 'hate': 0.78
Precision for 'nothate': 0.81
F1-Score for 'hate': 0.81
F1-Score for 'nothate': 0.76
bert-medium
